# Create a model for the repeat station network: Model 3

## Description

This model has two version Model 3a and Model 3b. They are a combination of the best aspects of Models 1 and 2.

The Model 3a is based on the ideia that the data reduction process is going to be done using current magnetic observatories in South America, the proposed new observatories and local variometers, as recommended by IAGA.
The Model 3b is based on the ideia that the data reduction process is going to be done using current magnetic observatories in South America, the proposed new observatories and central variometers, specially in the regions affected by the equatorial electrojet and SAMA.

This model contains:
- Current available Intermagnet Observatories in South America
- Current operational magnetic station (Macapá)
- The proposed new repeat stations network
- Proposed new magnetic stations to increase the geomagnetic coverage in the country (Their placement is not definitive, they are here just to give an idea)
- Proposed new observatories Tefe-AM, Pantanal-MT, Itacuruba-PE, and Sao Martinho da Serra-RS (Their placement is not definitive, they are here just to give an idea)
- EMBRACER stations

In [1]:
# Import modules
import mestrado_module as mm
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from folium import plugins
import seaborn as sns
from branca.element import Template, MacroElement

In [2]:
# Shapefile info for the static map
shapefile_folder: Path = Path(mm.path_south_america_shapefile)
shapefile_file: Path = Path(mm.south_america_shapefile)

# Magnetic Observatories
sa_obs_folder: Path = Path(mm.path_00_data_manual)
sa_obs_file: Path = Path("models_intermagnet_observatories_south_america.csv")

# Proposed magnetic observatories
prop_obs_folder: Path = Path(mm.path_00_data_manual)
prop_obs_file: Path = Path("models_proposed_observatories.csv")

# Magnetic stations
mag_sts_folder: Path = Path(mm.path_00_data_manual)
mag_sts_file: Path = Path("models_magnetic_stations_brazil.csv")

# Proposed magnetic stations
prop_mag_sts_folder: Path = Path(mm.path_00_data_manual)
prop_mag_sts_file: Path = Path("models_proposed_magnetic_stations.csv")

# EMBRACER stations
embracer_stations_folder: Path = Path(mm.path_00_data_manual)
embracer_stations_file: Path = Path("models_embracer_stations_all.csv")

# Selected repeat stations (folium file)
selected_rs_folder: Path = Path(mm.path_pipeline_07_select_rs_geo_distribution)
selected_rs_file: Path = Path(mm.output_7a_code_selected_rs_folium)

# Save figures and files
output_folder: Path = Path(mm.path_pipeline_10_model_three)

# Folium Maps
mapa0 = "../02_pipeline/10_rs_network_model_three/10a_model_three_a_obs_central_variometer_rs.html"
mapa1 = "../02_pipeline/10_rs_network_model_three/10b_model_three_b_obs_local_variometer_rs.html"

# Static maps
#static_map_no_radius: Path = Path("9_model_two_observatories_central_variometer_and_rs_no_radius.png")

# Figure style
sns.set_style("darkgrid")

## Read the data

In [3]:
# STATIONS DATA
selected_rs = pd.read_csv(selected_rs_folder /selected_rs_file)

# this has to be removed, otherwise folium wont read the geometry correctly later for some reason
# Therefore is better to create the geometry everytime it is needed
selected_rs = selected_rs.drop(columns=["geometry"]) 

# Check info
selected_rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Code                      50 non-null     object 
 1   Lat_dd                    50 non-null     float64
 2   Lon_dd                    50 non-null     float64
 3   Alt_m                     50 non-null     float64
 4   Time_dy                   50 non-null     float64
 5   D_dd                      50 non-null     float64
 6   IGRF_D_dd                 50 non-null     float64
 7   I_dd                      50 non-null     float64
 8   IGRF_I_dd                 50 non-null     float64
 9   F_nT                      50 non-null     float64
 10  IGRF_F_nT                 50 non-null     float64
 11  H_nT                      50 non-null     float64
 12  IGRF_H_nT                 50 non-null     float64
 13  X_nT                      50 non-null     float64
 14  IGRF_X_nT   

In [4]:
# INTERMAGNET OBSERVATORIES IN SOUTH AMERICA
south_america_obs_df = pd.read_csv(sa_obs_folder / sa_obs_file)

# Check df info
south_america_obs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Code        6 non-null      object 
 1   Name        6 non-null      object 
 2   Lat_dd      6 non-null      float64
 3   Lon_dd      6 non-null      float64
 4   Altitude_m  6 non-null      int64  
 5   Country     6 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 416.0+ bytes


In [5]:
# PROPOSED OBSERVATORIES
prop_obs_df = pd.read_csv(prop_obs_folder / prop_obs_file)

# Check df info
prop_obs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Code        4 non-null      object 
 1   Name        4 non-null      object 
 2   Lat_dd      4 non-null      float64
 3   Lon_dd      4 non-null      float64
 4   Altitude_m  4 non-null      int64  
 5   Country     4 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 320.0+ bytes


In [6]:
# PRESENT-DAY MAGNETIC STATION
mag_sts_df = pd.read_csv(mag_sts_folder / mag_sts_file)

# Check df info
mag_sts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Magnetic Station  1 non-null      object 
 1   Name              1 non-null      object 
 2   Lat_dd            1 non-null      float64
 3   Lon_dd            1 non-null      float64
 4   Altitude_m        1 non-null      int64  
 5   Status            1 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 176.0+ bytes


In [7]:
# PROPOSED MAGNETIC STATIONS
prop_mag_sts_df = pd.read_csv(prop_mag_sts_folder / prop_mag_sts_file)

# Check df info
prop_mag_sts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Magnetic Station  6 non-null      object 
 1   Name              6 non-null      object 
 2   Lat_dd            6 non-null      float64
 3   Lon_dd            6 non-null      float64
 4   Altitude_m        6 non-null      int64  
 5   Status            6 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 416.0+ bytes


In [8]:
# EMBRACER STATIONS
embracer_df = pd.read_csv(embracer_stations_folder / embracer_stations_file)

# active stations according to the website
embracer_active_df = embracer_df[embracer_df["Status_Site"] == "On"]
embracer_active_df.info()

# inactive stations according to the website
embracer_inactive_df = embracer_df[embracer_df["Status_Site"] == "Off"]
embracer_inactive_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 2 to 17
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Code          6 non-null      object 
 1   UN            6 non-null      object 
 2   Local         6 non-null      object 
 3   Lat_dd        6 non-null      float64
 4   Lon_dd        6 non-null      float64
 5   Lat_deg       6 non-null      int64  
 6   Lat_min       6 non-null      int64  
 7   Lat_s         6 non-null      int64  
 8   Lon_deg       6 non-null      int64  
 9   Lon_min       6 non-null      int64  
 10  Lon_s         6 non-null      int64  
 11  Altitude_m    6 non-null      int64  
 12  Status_Paper  6 non-null      object 
 13  Status_Site   6 non-null      object 
dtypes: float64(2), int64(7), object(5)
memory usage: 720.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 1 to 22
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype

## Create the geodataframe for the folium map

In [9]:
# Repeat stations data
geometry_selected_rs = [Point(xy) for xy in zip(selected_rs["Lon_dd"], selected_rs["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_selected_rs = gpd.GeoDataFrame(selected_rs, geometry = geometry_selected_rs)

# Check it
gdf_selected_rs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Code                      50 non-null     object  
 1   Lat_dd                    50 non-null     float64 
 2   Lon_dd                    50 non-null     float64 
 3   Alt_m                     50 non-null     float64 
 4   Time_dy                   50 non-null     float64 
 5   D_dd                      50 non-null     float64 
 6   IGRF_D_dd                 50 non-null     float64 
 7   I_dd                      50 non-null     float64 
 8   IGRF_I_dd                 50 non-null     float64 
 9   F_nT                      50 non-null     float64 
 10  IGRF_F_nT                 50 non-null     float64 
 11  H_nT                      50 non-null     float64 
 12  IGRF_H_nT                 50 non-null     float64 
 13  X_nT                      50 non-null     fl

In [10]:
# INTERMAGNET OBSERVATORIES IN SOUTH AMERICA
south_america_obs_geometry = [Point(xy) for xy in zip(south_america_obs_df["Lon_dd"], south_america_obs_df["Lat_dd"])]

# Create the geodataframe
gdf_south_america_obs = gpd.GeoDataFrame(south_america_obs_df, geometry = south_america_obs_geometry)
gdf_south_america_obs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Code        6 non-null      object  
 1   Name        6 non-null      object  
 2   Lat_dd      6 non-null      float64 
 3   Lon_dd      6 non-null      float64 
 4   Altitude_m  6 non-null      int64   
 5   Country     6 non-null      object  
 6   geometry    6 non-null      geometry
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 464.0+ bytes


In [11]:
# PROPOSED OBSERVATORIES
prop_obs_geometry = [Point(xy) for xy in zip(prop_obs_df["Lon_dd"], prop_obs_df["Lat_dd"])]

# Create the geodataframe
gdf_prop_obs = gpd.GeoDataFrame(prop_obs_df, geometry = prop_obs_geometry)
gdf_prop_obs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Code        4 non-null      object  
 1   Name        4 non-null      object  
 2   Lat_dd      4 non-null      float64 
 3   Lon_dd      4 non-null      float64 
 4   Altitude_m  4 non-null      int64   
 5   Country     4 non-null      object  
 6   geometry    4 non-null      geometry
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 352.0+ bytes


In [12]:
# PRESENT-DAY MAGNETIC STATION
mag_sts_geometry = [Point(xy) for xy in zip(mag_sts_df["Lon_dd"], mag_sts_df["Lat_dd"])]

# Create the geodataframe
gdf_mag_sts = gpd.GeoDataFrame(mag_sts_df, geometry = mag_sts_geometry)
gdf_mag_sts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Magnetic Station  1 non-null      object  
 1   Name              1 non-null      object  
 2   Lat_dd            1 non-null      float64 
 3   Lon_dd            1 non-null      float64 
 4   Altitude_m        1 non-null      int64   
 5   Status            1 non-null      object  
 6   geometry          1 non-null      geometry
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 184.0+ bytes


In [13]:
# PROPOSED MAGNETIC STATION
prop_mag_sts_geometry = [Point(xy) for xy in zip(prop_mag_sts_df["Lon_dd"], prop_mag_sts_df["Lat_dd"])]

# Create the geodataframe
gdf_prop_mag_sts = gpd.GeoDataFrame(prop_mag_sts_df, geometry = prop_mag_sts_geometry)
gdf_prop_mag_sts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Magnetic Station  6 non-null      object  
 1   Name              6 non-null      object  
 2   Lat_dd            6 non-null      float64 
 3   Lon_dd            6 non-null      float64 
 4   Altitude_m        6 non-null      int64   
 5   Status            6 non-null      object  
 6   geometry          6 non-null      geometry
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 464.0+ bytes


In [14]:
# EMBRACER STATIONS

# active stations according to the website
embracer_active_geometry = [Point(xy) for xy in zip(embracer_active_df["Lon_dd"], embracer_active_df["Lat_dd"])]
# Create the geodataframe
gdf_embracer_active = gpd.GeoDataFrame(embracer_active_df, geometry = embracer_active_geometry)
gdf_embracer_active.info()

# inactive stations according to the website
embracer_inactive_geometry = [Point(xy) for xy in zip(embracer_inactive_df["Lon_dd"], embracer_inactive_df["Lat_dd"])]
# Create the geodataframe
gdf_embracer_inactive = gpd.GeoDataFrame(embracer_inactive_df, geometry = embracer_inactive_geometry)
gdf_embracer_inactive.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6 entries, 2 to 17
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Code          6 non-null      object  
 1   UN            6 non-null      object  
 2   Local         6 non-null      object  
 3   Lat_dd        6 non-null      float64 
 4   Lon_dd        6 non-null      float64 
 5   Lat_deg       6 non-null      int64   
 6   Lat_min       6 non-null      int64   
 7   Lat_s         6 non-null      int64   
 8   Lon_deg       6 non-null      int64   
 9   Lon_min       6 non-null      int64   
 10  Lon_s         6 non-null      int64   
 11  Altitude_m    6 non-null      int64   
 12  Status_Paper  6 non-null      object  
 13  Status_Site   6 non-null      object  
 14  geometry      6 non-null      geometry
dtypes: float64(2), geometry(1), int64(7), object(5)
memory usage: 768.0+ bytes
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 10 e

## Combination of proposed magnetic stations, Online and Offline EMBRACER

In [15]:
# View original
#gdf_prop_mag_sts
#gdf_embracer_active
gdf_embracer_inactive

,Code,UN,Local,Lat_dd,Lon_dd,Lat_deg,Lat_min,Lat_s,Lon_deg,Lon_min,Lon_s,Altitude_m,Status_Paper,Status_Site,geometry
1,MAN,BR,Manaus,-2.888333,-59.969722,-2,53,18,-59,58,11,102,Funcionando,Off,POINT (-59.96972 -2.88833)
4,SLZ,BR,São Luis,-2.594167,-44.209722,-2,35,39,-44,12,35,32,Funcionando,Off,POINT (-44.20972 -2.59417)
5,ALF,BR,Alta Floresta,-9.870278,-56.104167,-9,52,13,-56,6,15,284,Funcionando,Off,POINT (-56.10417 -9.87028)
6,ARA,BR,Araguantis,-5.600278,-48.100556,-5,36,1,-48,6,2,103,Funcionando,Off,POINT (-48.10056 -5.60028)
7,EUS,BR,Eusebio,-3.880000,-38.424444,-3,52,48,-38,25,28,43,Funcionando,Off,POINT (-38.42444 -3.88000)
13,TCM,AR,Tucuman,-26.822222,-65.194444,-26,49,20,-65,11,40,431,Funcionando,Off,POINT (-65.19444 -26.82222)
14,MED,BR,Medianeira,-25.295278,-54.093889,-25,17,43,-54,5,38,402,Planejado,Off,POINT (-54.09389 -25.29528)
18,SMS,BR,Sao Martinho da Serra,-29.443611,-53.822778,-29,26,37,-53,49,22,462,Funcionando,Off,POINT (-53.82278 -29.44361)
19,CHI,CH,Chillan,-36.637500,-71.999167,-36,38,15,-71,59,57,124,Planejado,Off,POINT (-71.99917 -36.63750)
22,RGA,AR,Rio Grande,-53.785833,-67.761667,-53,47,9,-67,45,42,10,Funcionando,Off,POINT (-67.76167 -53.78583)


In [16]:
# Removing the unecessary
best_prop_mag_sts = gdf_prop_mag_sts.drop([1])
best_on_embracer = gdf_embracer_active.drop([9,16,17])
best_off_embracer = gdf_embracer_inactive.drop([13,18,19,22])

In [17]:
# Test
best_on_embracer

,Code,UN,Local,Lat_dd,Lon_dd,Lat_deg,Lat_min,Lat_s,Lon_deg,Lon_min,Lon_s,Altitude_m,Status_Paper,Status_Site,geometry
2,PVE,BR,Porto velho,-8.763611,-63.906389,-8,45,49,-63,54,23,83,Planejado,On,POINT (-63.90639 -8.76361)
11,JAT,BR,Jatai,-17.931944,-51.718056,-17,55,55,-51,43,5,679,Funcionando,On,POINT (-51.71806 -17.93194)
15,CXP,BR,Cachoeira Paulista,-22.068611,-45.014444,-22,4,7,-45,0,52,601,Funcionando,On,POINT (-45.01444 -22.06861)


## Plot folium map

In [18]:
## DEFINITIONS (distances in km)
#observatory_coverage_radius = 904000 # BASED ON ITALY CASTELLO TESINO
#embracer_coverage_radius = 150000 # germany
#embracer_coverage_radius = 230000 # italy GIB station

# Radius suggested by Katia and Cristiano (800 km, 300 km)
observatory_coverage_radius_m = 800000
repeat_station_coverage_radius_m = 300000
magnetic_station_coverage_radius_m = 500000

# Color definitions
sa_obs_color = "darkblue"
prop_obs_color = "darkred"
selected_rs_color = "black"
mag_sts_color = "pink"
prop_mag_sts_color = "purple"
embracer_active_color = "green"
embracer_inactive_color = "gray"

# Custom icons for the repeat stations and observatories
icon_path_sa_obs = r"../00_data/manual/symbols_square_darkblue.png"
icon_path_prop_obs = r"../00_data/manual/symbols_square_darkred.png"
icon_path_rs = r"../00_data/manual/symbols_circle_black.png"
icon_path_mag_sts = r"../00_data/manual/symbols_triangle_pink.png"
icon_path_prop_mag_sts = r"../00_data/manual/symbols_triangle_purple.png"
icon_path_embracer_active = r"../00_data/manual/symbols_triangle_green.png"
icon_path_embracer_inactive = r"../00_data/manual/symbols_triangle_gray.png"

### Model 3a: Magnetic observatories, propositions and central variometers

In [19]:
# INTERACTIVE MAP: Map of the Model 1, current observatories and proposed observatories

# Create the Interactive map using folium
tile_type0 = "OpenStreetMap"
map0 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type0)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Selected Repeat Stations
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, active radius', show = False)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Current closest brazilian observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g01)
# Add the repeat stations coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color #,
        #fill_color = "green"    
    ).add_to(g01)


# Group 2: Selected Repeat Stations
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, inactive radius', show = True)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Current closest brazilian observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g02)    


# Group 3: Magnetic Observatories in South America
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America', show = True)
for index, location_info in gdf_south_america_obs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g03)
# Add the observatory coverage    
for index, location_info in gdf_south_america_obs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = sa_obs_color,
        fill_color = sa_obs_color
    ).add_to(g03)
        
        
# Group 4: Proposed Magnetic Observatories in brazil
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Proposed Magnetic Observatories', show = True)
for index, location_info in gdf_prop_obs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_prop_obs, icon_size=(14, 14)),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g04)
# Add the observatory coverage    
for index, location_info in gdf_prop_obs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = prop_obs_color,
        fill_color = prop_obs_color
    ).add_to(g04)     
        
        
# Group 5: Magnetic Stations
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Stations', show = True)
for index, location_info in gdf_mag_sts.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_mag_sts, icon_size=(14, 14)),
        tooltip = [
            "Magnetic Station:",
            location_info["Magnetic Station"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Name"],
            "Status",
            location_info["Status"],
        ],
    ).add_to(g05)
# Add the magnetic station coverage    
for index, location_info in gdf_mag_sts.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = magnetic_station_coverage_radius_m,
        color = mag_sts_color,
        fill_color = mag_sts_color
    ).add_to(g05) 

#=============================================================
# GROUPS WITH THE BEST OF EACH GROUP

# Group 6: Best of Proposed Magnetic Stations ACTIVE RADIUS
g06 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Proposed Magnetic Stations, active radius', show = False)
for index, location_info in best_prop_mag_sts.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_prop_mag_sts, icon_size=(14, 14)),
        tooltip = [
            "Magnetic Station:",
            location_info["Magnetic Station"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Name"],
            "Status",
            location_info["Status"],
        ],
    ).add_to(g06)
# Add the magnetic station coverage    
for index, location_info in best_prop_mag_sts.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = magnetic_station_coverage_radius_m,
        color = prop_mag_sts_color,
        fill_color = prop_mag_sts_color
    ).add_to(g06)
        
# Group 7: Best of Proposed Magnetic Stations INACTIVE RADIUS
g07 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Proposed Magnetic Stations, inactive radius', show = True)
for index, location_info in best_prop_mag_sts.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_prop_mag_sts, icon_size=(14, 14)),
        tooltip = [
            "Magnetic Station:",
            location_info["Magnetic Station"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Name"],
            "Status",
            location_info["Status"],
        ],
    ).add_to(g07)
        

# Group 8: Best of Online EMBRACER Magnetic Stations ACTIVE RADIUS
g08 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Online EMBRACER Magnetic Stations, active radius', show = False)
for index, location_info in best_on_embracer.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_embracer_active, icon_size=(14, 14)),
        tooltip = [
            "EMBRACER:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["UN"],
            "Status",
            location_info["Status_Site"],
        ],
    ).add_to(g08)
# Add the magnetic station coverage    
for index, location_info in best_on_embracer.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = magnetic_station_coverage_radius_m,
        color = embracer_active_color,
        fill_color = embracer_active_color
    ).add_to(g08)

    
# Group 8: Best of Online EMBRACER Magnetic Stations INACTIVE RADIUS
g09 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Online EMBRACER Magnetic Stations, inactive radius', show = True)
for index, location_info in best_on_embracer.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_embracer_active, icon_size=(14, 14)),
        tooltip = [
            "EMBRACER:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["UN"],
            "Status",
            location_info["Status_Site"],
        ],
    ).add_to(g09)
 
        
# Group 10: Best of Offline EMBRACER Magnetic Stations ACTIVE RADIUS
g10 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Offline EMBRACER Magnetic Stations, active radius', show = False)
for index, location_info in best_off_embracer.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_embracer_inactive, icon_size=(14, 14)),
        tooltip = [
            "EMBRACER:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["UN"],
            "Status",
            location_info["Status_Site"],
        ],
    ).add_to(g10)
# Add the magnetic station coverage    
for index, location_info in best_off_embracer.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = magnetic_station_coverage_radius_m,
        color = embracer_inactive_color,
        fill_color = embracer_inactive_color
    ).add_to(g10)
        

# Group 11: Best of Offline EMBRACER Magnetic Stations INACTIVE RADIUS
g11 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Best of Offline EMBRACER Magnetic Stations, inactive radius', show = True)
for index, location_info in best_off_embracer.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_embracer_inactive, icon_size=(14, 14)),
        tooltip = [
            "EMBRACER:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["UN"],
            "Status",
            location_info["Status_Site"],
        ],
    ).add_to(g11)
        
# Add the subgroups to the main map
map0.add_child(fg0)
map0.add_child(g01)
map0.add_child(g02)
map0.add_child(g03)
map0.add_child(g04)
map0.add_child(g05)
map0.add_child(g06)
map0.add_child(g07)
map0.add_child(g08)
map0.add_child(g09)
map0.add_child(g10)
map0.add_child(g11)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map0)
folium.TileLayer('Stamen Toner').add_to(map0)
#folium.TileLayer('Stamen Water Color').add_to(map0)
folium.TileLayer('cartodbpositron').add_to(map0)
#folium.TileLayer('cartodbdark_matter').add_to(map0)

# Add the layer control
folium.LayerControl().add_to(map0)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map0.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map0.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map0.add_child(fullscreen_button0)

# Add a mini map
minimap0 = plugins.MiniMap()
map0.add_child(minimap0)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map0.add_child(draw0)


# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Model 3a: Magnetic observatories, Central variometer and Repeat Stations</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Repeat Stations</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>
    <li><span style='background:pink;opacity:0.7;'></span>Magnetic Station</li>
    <li><span style='background:darkred;opacity:0.7;'></span>Proposed Magnetic Observatories</li>
    <li><span style='background:purple;opacity:0.7;'></span>Best of Proposed Magnetic Stations</li>
    <li><span style='background:green;opacity:0.7;'></span>Best of Online EMBRACER Stations</li>
    <li><span style='background:gray;opacity:0.7;'></span>Best of Offline EMBRACER Stations</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map0.get_root().add_child(macro)


# Save map
map0.save(mapa0)

### Model 3b: Magnetic observatories, propositions and local variometers

In [20]:
# INTERACTIVE MAP: Map of the Model 1, current observatories and proposed observatories

# Create the Interactive map using folium
tile_type0 = "OpenStreetMap"
map1 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type0)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Selected Repeat Stations
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, active radius', show = False)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Current closest brazilian observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g01)
# Add the repeat stations coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color #,
        #fill_color = "green"    
    ).add_to(g01)


# Group 2: Selected Repeat Stations
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Repeat Stations, inactive radius', show = True)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Current closest brazilian observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g02)    


# Group 3: Magnetic Observatories in South America
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America', show = True)
for index, location_info in gdf_south_america_obs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g03)
# Add the observatory coverage    
for index, location_info in gdf_south_america_obs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = sa_obs_color,
        fill_color = sa_obs_color
    ).add_to(g03)
        
        
# Group 4: Proposed Magnetic Observatories in brazil
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Proposed Magnetic Observatories', show = True)
for index, location_info in gdf_prop_obs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_prop_obs, icon_size=(14, 14)),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g04)
# Add the observatory coverage    
for index, location_info in gdf_prop_obs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = prop_obs_color,
        fill_color = prop_obs_color
    ).add_to(g04)     
        
        
# Group 5: Magnetic Stations
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Stations', show = True)
for index, location_info in gdf_mag_sts.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        icon = folium.Icon(color = prop_obs_color, icon ="square", prefix = "fa"),
        icon = folium.features.CustomIcon(icon_path_mag_sts, icon_size=(14, 14)),
        tooltip = [
            "Magnetic Station:",
            location_info["Magnetic Station"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Name"],
            "Status",
            location_info["Status"],
        ],
    ).add_to(g05)
# Add the magnetic station coverage    
for index, location_info in gdf_mag_sts.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = magnetic_station_coverage_radius_m,
        color = mag_sts_color,
        fill_color = mag_sts_color
    ).add_to(g05) 

        
# Add the subgroups to the main map
map1.add_child(fg0)
map1.add_child(g01)
map1.add_child(g02)
map1.add_child(g03)
map1.add_child(g04)
map1.add_child(g05)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map1)
folium.TileLayer('Stamen Toner').add_to(map1)
#folium.TileLayer('Stamen Water Color').add_to(map1)
folium.TileLayer('cartodbpositron').add_to(map1)
#folium.TileLayer('cartodbdark_matter').add_to(map1)

# Add the layer control
folium.LayerControl().add_to(map1)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map1.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map1.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map1.add_child(fullscreen_button0)

# Add a mini map
minimap1 = plugins.MiniMap()
map1.add_child(minimap1)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map1.add_child(draw0)


# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Model 3b: Magnetic observatories, local variometer and Repeat Stations</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Repeat Stations</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>
    <li><span style='background:pink;opacity:0.7;'></span>Magnetic Station</li>
    <li><span style='background:darkred;opacity:0.7;'></span>Proposed Magnetic Observatories</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map1.get_root().add_child(macro)


# Save map
map1.save(mapa1)